In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scipy
from matplotlib import rc
import matplotlib
import yfinance
import seaborn as sns
plt.style.use('seaborn')
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=["grey", "green", "black", "deepskyblue", "orange", "lime", "purple", "Gold", "brown", "blue", "pink"]) 

In [ ]:
assets = {"EEM": "Emerging Markets",
           "IVV": "S&P 500",
           "IEV": "MSCI Europe",
           "IXN": "Global Tech",
           "IYR": "US Real Estate",
           "IXG": "Global Financials",
           "EXI": "Global Industrials",
           "GC=F": "Gold Futures", 
           "BZ=F": "Brent Crude Oil Futures",
           "HYG": "High-Yield Corporate Bonds",
           "TLT": "20+ Year Treasury Bonds"}
tickers = list(assets.keys())
asset_names = list(assets.values())
#spx = pd.read_csv('../data/spx.csv').stack().tolist()

**Downloading data and calculating returns**

In [ ]:
raw_data = yfinance.download(tickers, start="2008-1-1", back_adjust=True, auto_adjust=True)
closing_prices = raw_data['Close']
closing_prices = closing_prices[tickers]
returns = closing_prices.pct_change().iloc[1:].ffill()*100 #returns
index = closing_prices/closing_prices.iloc[0]*100 #index

**Plotter index figur med alle aktiver**

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=190)
ax.plot(index, label=asset_names, linewidth=1)
ax.legend()
fig.savefig("Stocks_index_plot.png", bbox_inches = 'tight', pad_inches = 0.1)

**Fitter fordelinger og laver PDF'er for udvalgte tickers, samt plotter PDF med real world data**

In [ ]:
ticker_list=["IVV","GC=F","HYG"]

for ticker in ticker_list:
    mu, std = scipy.norm.fit(returns[ticker]) #normal distribution fi#t
    beta, loc_ged, scale_ged = scipy.gennorm.fit(returns[ticker]) #GED f#it
    df, nc_t, loc_t, scale_t = scipy.nct.fit(returns[ticker]) #skewed student's t distrubiton fit

    #draw data
    xmin = returns[ticker].min()
    xmax = returns[ticker].max()
    x = np.linspace(xmin, xmax, 1000)

    normal_data=scipy.norm.pdf(x, mu, std)
    GED_data=scipy.gennorm.pdf(x, beta,loc_ged,scale_ged)
    t_data=scipy.nct.pdf(x, df,nc_t,loc_t,scale_t)
    
    plt.style.use('seaborn')
    fig, ax = plt.subplots(1, 1, figsize=(7, 7), dpi=190)
    ax.plot(x, normal_data, label="Fitted gaussian")#, c="blue")
    ax.plot(x, GED_data, label="Fitted GED")#, c = "black")
    ax.plot(x, t_data, label ="Fitted skewed-t")#, c="red")
    ax.hist(returns[ticker], bins=100, alpha=0.6, density = True)
    ax.set_xlim(-7.5, 7.5)
    ax.legend()
    ax.set_xlabel('Returns') 
    ax.set_ylabel('Probability Density',loc='top', rotation = 0, labelpad = -110) 
    fig.savefig(f"Density_{ticker}.png", bbox_inches = 'tight', pad_inches = 0.1)

Fitter GARCH model og plotter data

In [ ]:
import arch
for ticker in ticker_list:
    #set up model:
    dist = arch.univariate.SkewStudent()
    vol = arch.univariate.GARCH(p=1, o=1, q=1)
    model = arch.univariate.ConstantMean(returns[ticker], volatility=vol, distribution=dist)
    #estimate model
    results = model.fit(disp = 'off')
    params = results.params
    #simulate data
    sim_returns = model.simulate(params, len(returns[ticker])).iloc[:,0]
    #plot simulated data
    f, ax1 = plt.subplots(1, 1, figsize=(15,12))
    ax1.plot(sim_returns, alpha=.8 , label="Simulated")
    ax1.plot(returns[ticker].values, alpha=.8,label="Real Data")
    ax1.set_ylabel('Daily Returns',loc='top', rotation = 0, labelpad = -100)
    ax1.legend()
    ax1.set_ylim((-50,50))
    #fig.savefig(f"Garch_timeseries_{ticker}.png", bbox_inches = 'tight', pad_inches = 0.1)

**Laver Jarque bera normalitets test**

In [ ]:
mean = returns.mean()
std = returns.std()
#normality test
jarque = returns.apply(scipy.jarque_bera, axis=0)
p_values = jarque.iloc[1]
#skewed students t parameters
t_dist= returns.apply(scipy.nct.fit, axis=0) 
df_t=t_dist.iloc[0]
nc_t=t_dist.iloc[1]
loc_t=t_dist.iloc[2]
scale_t=t_dist.iloc[3]

row_names = pd.Index(['Mean', 'Std', 'P_value Jarque', 'Degrees of freedom','Non central','Location','Scale'])
pd.DataFrame([mean, std, p_values, df_t, nc_t, loc_t, scale_t], columns = tickers, index = row_names)

**Laver cool covariance matrix**

In [ ]:
cov = returns.cov()
cov.style.background_gradient(cmap='afmhot', axis=None).set_precision(2)